# Домашнее задание 2. Классификация изображений.

В этом задании потребуется обучить классификатор изображений. Будем работать с датасетом, название которого раскрывать не будем. Можете посмотреть самостоятельно на картинки, которые в есть датасете. В нём 200 классов и около 5 тысяч картинок на каждый класс. Классы пронумерованы, как нетрудно догадаться, от 0 до 199. Скачать датасет можно вот [тут](https://yadi.sk/d/BNR41Vu3y0c7qA).

Структура датасета простая -- есть директории train/ и val/, в которых лежат обучающие и валидационные данные. В train/ и val/ лежат директориии, соответствующие классам изображений, в которых лежат, собственно, сами изображения.
 
__Задание__. Необходимо выполнить любое из двух заданий

1) Добейтесь accuracy **на валидации не менее 0.44**. В этом задании **запрещено** пользоваться предобученными моделями и ресайзом картинок. 

2) Добейтесь accuracy **на валидации не менее 0.84**. В этом задании делать ресайз и использовать претрейн можно. 

Напишите краткий отчёт о проделанных экспериментах. Что сработало и что не сработало? Почему вы решили, сделать так, а не иначе? Обязательно указывайте ссылки на чужой код, если вы его используете. Обязательно ссылайтесь на статьи / блогпосты / вопросы на stackoverflow / видосы от ютуберов-машинлернеров / курсы / подсказки от Дяди Васи и прочие дополнительные материалы, если вы их используете. 

Ваш код обязательно должен проходить все `assert`'ы ниже.

Необходимо написать функции `train_one_epoch`, `train` и `predict` по шаблонам ниже (во многом повторяют примеры с семинаров).Обратите особое внимание на функцию `predict`: она должна возвращать список лоссов по всем объектам даталоадера, список предсказанных классов для каждого объекта из даталоалера и список настоящих классов для каждого объекта в даталоадере (и именно в таком порядке).

__Использовать внешние данные для обучения строго запрещено в обоих заданиях. Также запрещено обучаться на валидационной выборке__.


__Критерии оценки__: Оценка вычисляется по простой формуле: `min(10, 10 * Ваша accuracy / 0.44)` для первого задания и `min(10, 10 * (Ваша accuracy - 0.5) / 0.34)` для второго. Оценка округляется до десятых по арифметическим правилам. Если вы выполнили оба задания, то берется максимум из двух оценок.

__Бонус__. Вы получаете 5 бонусных баллов если справляетесь с обоими заданиями на 10 баллов (итого 15 баллов). В противном случае выставляется максимальная из двух оценок и ваш бонус равен нулю.

__Советы и указания__:
 - Наверняка вам потребуется много гуглить о классификации и о том, как заставить её работать. Это нормально, все гуглят. Но не забывайте, что нужно быть готовым за скатанный код отвечать :)
 - Используйте аугментации. Для этого пользуйтесь модулем `torchvision.transforms` или библиотекой [albumentations](https://github.com/albumentations-team/albumentations)
 - Можно обучать с нуля или файнтюнить (в зависимости от задания) модели из `torchvision`.
 - Рекомендуем написать вам сначала класс-датасет (или воспользоваться классом `ImageFolder`), который возвращает картинки и соответствующие им классы, а затем функции для трейна по шаблонам ниже. Однако делать это мы не заставляем. Если вам так неудобно, то можете писать код в удобном стиле. Однако учтите, что чрезмерное изменение нижеперечисленных шаблонов увеличит количество вопросов к вашему коду и повысит вероятность вызова на защиту :)
 - Валидируйте. Трекайте ошибки как можно раньше, чтобы не тратить время впустую.
 - Чтобы быстро отладить код, пробуйте обучаться на маленькой части датасета (скажем, 5-10 картинок просто чтобы убедиться что код запускается). Когда вы поняли, что смогли всё отдебажить, переходите обучению по всему датасету
 - На каждый запуск делайте ровно одно изменение в модели/аугментации/оптимайзере, чтобы понять, что и как влияет на результат.
 - Фиксируйте random seed.
 - Начинайте с простых моделей и постепенно переходите к сложным. Обучение лёгких моделей экономит много времени.
 - Ставьте расписание на learning rate. Уменьшайте его, когда лосс на валидации перестаёт убывать.
 - Советуем использовать GPU. Если у вас его нет, используйте google colab. Если вам неудобно его использовать на постоянной основе, напишите и отладьте весь код локально на CPU, а затем запустите уже написанный ноутбук в колабе. Авторское решение задания достигает требуемой точности в колабе за 15 минут обучения.
 
Good luck & have fun! :)

In [1]:
!pip install wandb --upgrade

     |████████████████████████████████| 1.7 MB 909 kB/s eta 0:00:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.12.5
    Uninstalling wandb-0.12.5:
      Successfully uninstalled wandb-0.12.5


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tqdm
from tqdm.auto import tqdm
import wandb

import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score

import os
from os.path import isfile, join
import sys
import glob
import copy
import random
import pprint

import PIL
from PIL import Image

### Подготовка данных

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [3]:
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  


wandb: ERROR No API key specified.
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
# Tnx to https://medium.com/analytics-vidhya/creating-a-custom-dataset-and-dataloader-in-pytorch-76f210a1df5d  for deeper understanding

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform):
        self.imgs_path = os.path.expanduser(data_dir)
        self.transform = transform

        file_list = glob.glob(self.imgs_path + "*")

        self.data = []
        for class_path in file_list:
            class_name = int(class_path.split("/")[-1][-3:])
            for img_path in glob.glob(class_path + "/*.jpg"):
                self.data.append([img_path, class_name])
        self.img_dim = (416, 416)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, class_name = self.data[idx]
        
        img = Image.open(img_path)
        img = img.convert('RGB')
        
        if self.transform is not None:
            img = self.transform(img)
          
        return img, class_name


In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [7]:
#train_transform = None
#val_transform = None

'''
# Normalisaion parameters for efficientnet version1:
mean = torch.tensor([0.05438065, 0.05291743, 0.07920227])
std = torch.tensor([0.39414383, 0.33547948, 0.38544176])

size = 300


# Normalisaion parameters for resnet:
mean = torch.tensor([0.485, 0.456, 0.406])
std = torch.tensor([0.229, 0.224, 0.225])

size = 224
'''

train_transform_basic = transforms.Compose(
    [transforms.RandomCrop(32),
     transforms.Resize((64,64)),
     transforms.ColorJitter(hue=.05, saturation=.05),
     transforms.RandomHorizontalFlip(),
     transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

'''
# If we want to use pretrained models we have to normalize data as following
# source: https://towardsdatascience.com/pytorch-ignite-classifying-tiny-imagenet-with-efficientnet-e5b1768e5e8f)

train_transform_for_pretrained = transforms.Compose([
                transforms.Resize((size + 4, size + 4)),
                transforms.CenterCrop(size), # Center crop image
                #transforms.RandomRotation(40),
                #transforms.RandomAffine(
                #    degrees=10,
                #    translate=(0.01, 0.12),
                #    shear=(0.01, 0.03)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),  # Converting cropped images to tensors
                transforms.Normalize(mean, std)
])

val_transform_for_pretrained = transforms.Compose([
                transforms.Resize((size, size)),
                transforms.ToTensor(),  # Converting cropped images to tensors
                transforms.Normalize(mean, std)
])


"./dataset/dataset/train/" for jupiter
/content/gdrive/MyDrive/ColabNotebooks/dataset/dataset/train/ for colab

'''

train_dataset = MyDataset(data_dir = "/kaggle/input/hw2-dataset/dataset/dataset/train/", transform = train_transform_basic)
val_dataset = MyDataset(data_dir = "/kaggle/input/hw2-dataset/dataset/dataset/val/", transform = train_transform_basic)


/opt/conda/lib/python3.7/site-packages/torchvision/transforms/transforms.py:1231: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"


In [8]:
def data_loaders(bs):
    
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=4)
    test_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=bs, shuffle=False, num_workers=4)
    
    return train_dataloader, test_dataloader

In [9]:
# Just very simple sanity checks
assert isinstance(train_dataset[0], tuple)
assert len(train_dataset[0]) == 2
assert isinstance(train_dataset[1][1], int)
print("tests passed")

tests passed


### Вспомогательные функции, реализация модели

In [10]:
# LR scheduler class - to adjust lr if loss doesn't change for a few epochs
# Early stopping class - to stop training if loss doesn't change for a long time

# Source: https://debuggercafe.com/using-learning-rate-scheduler-and-early-stopping-with-pytorch/ 

class LRScheduler():
    """
    Learning rate scheduler. If the validation loss does not decrease for the 
    given number of `patience` epochs, then the learning rate will decrease by
    by given `factor`.
    """
    def __init__(
        self, optimizer, patience=5, min_lr=1e-6, factor=0.5
    ):
        """
        new_lr = old_lr * factor

        :param optimizer: the optimizer we are using
        :param patience: how many epochs to wait before updating the lr
        :param min_lr: least lr value to reduce to while updating
        :param factor: factor by which the lr should be updated
        """
        self.optimizer = optimizer
        self.patience = patience
        self.min_lr = min_lr
        self.factor = factor

        self.lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau( 
                self.optimizer,
                mode='min',
                patience=self.patience,
                factor=self.factor,
                min_lr=self.min_lr,
                verbose=True
            )

    def __call__(self, val_loss):
        self.lr_scheduler.step(val_loss)

# I decided not to use early stopping as I was training my models only for 10 epochs 
'''
class EarlyStopping():
    """
    Early stopping to stop the training when the loss does not improve after
    certain epochs.
    """
    def __init__(self, patience=10, min_delta=0):
        """
        :param patience: how many epochs to wait before stopping when loss is
               not improving
        :param min_delta: minimum difference between new loss and old loss for
               new loss to be considered as an improvement
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss

            # Reset counter if validation loss improves
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            print(f"INFO: Early stopping counter {self.counter} of {self.patience}")

            if self.counter >= self.patience:
                print('INFO: Early stopping')
                self.early_stop = True
'''

'\nclass EarlyStopping():\n    """\n    Early stopping to stop the training when the loss does not improve after\n    certain epochs.\n    """\n    def __init__(self, patience=10, min_delta=0):\n        """\n        :param patience: how many epochs to wait before stopping when loss is\n               not improving\n        :param min_delta: minimum difference between new loss and old loss for\n               new loss to be considered as an improvement\n        """\n        self.patience = patience\n        self.min_delta = min_delta\n        self.counter = 0\n        self.best_loss = None\n        self.early_stop = False\n\n    def __call__(self, val_loss):\n        if self.best_loss == None:\n            self.best_loss = val_loss\n        elif self.best_loss - val_loss > self.min_delta:\n            self.best_loss = val_loss\n\n            # Reset counter if validation loss improves\n            self.counter = 0\n        elif self.best_loss - val_loss < self.min_delta:\n            se

In [18]:
# Functions from seminar #6

def train_one_epoch(
    model,
    train_dataloader,
    optimizer,
    criterion,
    return_losses=False,
    device="cuda:0",
):
    seed_everything(13)
    model = model.to(device).train()
    total_loss = 0
    num_batches = 0
    all_losses = []
    total_predictions = np.array([])#.reshape((0, ))
    total_labels = np.array([])#.reshape((0, ))
    with tqdm(total=len(train_dataloader), file=sys.stdout) as prbar:
        for images, labels in train_dataloader:
          
            # Move Batch to GPU
            images = images.to(device)
            labels = labels.to(device)
            predicted = model(images)
            loss = criterion(predicted, labels)

            # Update weights
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # Update descirption for tqdm
            accuracy = (predicted.argmax(1) == labels).float().mean()
            wandb.log({"batch loss": loss.item()})
            wandb.log({"batch accuracy": accuracy.item() * 100})
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)

            total_loss += loss.item()
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1
            all_losses.append(loss.detach().item())

    metrics = {"loss": total_loss / num_batches}
    metrics.update({"accuracy": (total_predictions == total_labels).mean()})

    if return_losses:
        return metrics, all_losses
    else:
        return metrics


def predict(model, val_dataloder, criterion, device="cuda:0"):

    seed_everything(13)
    model = model.eval()
    total_loss = []
    num_batches = 0
    total_predictions = np.array([])
    total_labels = np.array([])

    # New tqdm run
    with tqdm(total=len(val_dataloder), file=sys.stdout) as prbar:
        for images, labels in val_dataloder:
            images = images.to(device)
            labels = labels.to(device)
            predicted = model(images)
            loss = criterion(predicted, labels)
            accuracy = (predicted.argmax(1) == labels).float().mean()
            prbar.set_description(
                f"Loss: {round(loss.item(), 4)} "
                f"Accuracy: {round(accuracy.item() * 100, 4)}"
            )
            prbar.update(1)
            total_loss.append(loss.item())
            total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
            total_labels = np.append(total_labels, labels.cpu().detach().numpy())
            num_batches += 1

    metrics = {"loss_avg": sum(total_loss) / num_batches}
    metrics.update({"accuracy_avg": (total_predictions == total_labels).mean()})

    all_losses = total_loss
    predicted_classes = total_predictions
    true_classes = total_labels
    
    return metrics, all_losses, predicted_classes, true_classes

In [23]:
# commented cells are for normal evaluation, uncommented are for wandb run

def train(config=None):

    # Initialize a new wandb run - after finding the best parameters -> mode = "disabled" to train without wandb
    with wandb.init(config=config, mode = "disabled"):
        config = wandb.config

        device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
        criterion = nn.CrossEntropyLoss()

        model = get_model(device, config.dropout_value, config.dropout_2, config.dropout_4,
                        config.dropout_6, config.batchnorm_3, config.batchnorm_5,
                        config.n_features1, config.n_features2)
        '''
        model = get_model(device = device, dropout_value = 0.3, dropout_2 = False, dropout_4 = False,
                          dropout_6 = False, batchnorm_3 = True, batchnorm_5 = True, n_features1 = 256, n_features2 = 128)
        
        '''
      # Initializing out parameters from config, constructing model
        
        train_dataloader, val_dataloader = data_loaders(bs = config.batch_size)

        '''
        train_dataloader, val_dataloader = data_loaders(bs = 32)
        optimizer = build_optimizer(model, "adam", 0.0001)
        lr_scheduler = LRScheduler(optimizer)
        '''
        #early_stopping = EarlyStopping()
        
        seed_everything(13)
        all_train_losses = []
        epoch_train_losses = []
        epoch_eval_losses = []
        printing = False # in order to avoid a lot of printing on a screen after setting up parameters
        

        for epoch in range(config.epochs):
          if printing:
            print(f"Train Epoch: {epoch}")
          train_metrics, one_epoch_train_losses = train_one_epoch(
              model=model,
              train_dataloader=train_dataloader,
              optimizer=optimizer,
              return_losses=True,
              criterion=criterion,
              device=device
            )
          # Save Train losses
          all_train_losses.extend(one_epoch_train_losses)
          epoch_train_losses.append(train_metrics["loss"])

          # Eval step
          if printing:
                print(f"Validation Epoch: {epoch}")

          with torch.no_grad():
             validation_metrics = predict(
                  model=model,
                  val_dataloder=val_dataloader,
                  criterion=criterion)
            
          # Unproductivity check 
          lr_scheduler(val_epoch_loss)
            
          #early_stopping(val_epoch_loss)
          #if early_stopping.early_stop:
          #  break
        
          # Add metrics to wandb
          wandb.log({"mean val loss": validation_metrics[0]["loss_avg"],
                      "mean val accuracy": validation_metrics[0]["accuracy_avg"],
                      "mean train accuracy": train_metrics["accuracy"],
                      "mean train loss": train_metrics["loss"],
                      "epoch" : epoch})

           # Save eval losses
          epoch_eval_losses.append(validation_metrics[0]["loss_avg"])
          if printing:
                print(f"Validation loss (mean per batch): {validation_metrics[0]['loss_avg']}")

In [12]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer

### Mодели

In [13]:
class Net(torch.nn.Module):
    def __init__(self, dropout_value, dropout_2, dropout_4, dropout_6, batchnorm_3,
                 batchnorm_5, n_features1, n_features2):

        super().__init__()
        seed_everything(13)

        self.batch_norm1 = torch.nn.BatchNorm2d(3)
        # First conv layer
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            torch.nn.ReLU())

        # Second conv layer
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2),
        )
        if dropout_2:
            self.dropout2 = torch.nn.Dropout(p=0.2)
        else:
          self.dropout2 = torch.nn.Dropout(p=0)

        # Third conv layer
        if batchnorm_3:
            self.conv3 = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
                torch.nn.BatchNorm2d(32),
                torch.nn.ReLU())
        
        else:
            self.conv3 = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3),
                torch.nn.ReLU())
        
        # Fourth conv layer
        self.conv4 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2),
        )
          
        if dropout_4:
            self.dropout4 = torch.nn.Dropout(p=max(0.2, dropout_value - 0.2))
        else:
          self.dropout4 = torch.nn.Dropout(p=0)

        # Fifth conv layer
        if batchnorm_5:
            self.conv5 = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
                torch.nn.BatchNorm2d(64),
                torch.nn.ReLU())
        else:
            self.conv5 = torch.nn.Sequential(
                torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
                torch.nn.ReLU())

        
        # Sixth conv layer
        self.conv6 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2),
        )

        if dropout_6:
            self.dropout6 = torch.nn.Dropout(p=dropout_value)
        else:
          self.dropout6 = torch.nn.Dropout(p=0)

        # Linear layers
        self.linear1 = torch.nn.Linear(in_features=1024, out_features=n_features1)
        self.linear2 = torch.nn.Linear(in_features=n_features1, out_features=n_features2)
        self.output = torch.nn.Linear(in_features=n_features2, out_features=200)
        self.dropout_l = torch.nn.Dropout(p=dropout_value)


    def forward(self, x):

        x = self.batch_norm1(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.dropout2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.dropout4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.dropout6(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.linear1(x))
        x = self.dropout_l(x)
        x = F.relu(self.linear2(x))
        x = self.output(x)
        
        return x


In [14]:
def get_model(device, dropout_value, dropout_2, dropout_4, dropout_6, batchnorm_3,
              batchnorm_5, n_features1, n_features2):
    seed_everything(13)
    model = Net(dropout_value, dropout_2, dropout_4, dropout_6, batchnorm_3, batchnorm_5, n_features1, n_features2)
    model = model.to(device)
    return model

### Параметры и тд

In [15]:
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

metric = {
    'name': 'mean train loss',
    'goal': 'minimize'   
    }

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'n_features1': {
        'values': [128, 256, 512]
        },
    'n_features2': {
        'values': [128, 256, 512]
        },
    'dropout_value': {
          'values': [0.3, 0.4, 0.5, 0.6]
        },
    'dropout_2': {
          'values': [True, False]
        },
    'dropout_4': {
          'values': [True, False]
        },
    'dropout_6': {
          'values': [True, False]
        },
    'batchnorm_3': {
          'values': [True, False]
        },
    'batchnorm_5': {
          'values': [True, False]
        },
    'epochs': {
        'value': 10
        },
    'learning_rate': {
        'values': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05]
      },
    'batch_size': {
        'values': [16, 32, 64, 128, 256],
      }
    }

sweep_config = {
    'method': 'random',
    'metric': metric,
    'parameters': parameters_dict,
    }

pprint.pprint(sweep_config)

sweep_id = wandb.sweep(sweep_config, project="hw2")

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'mean train loss'},
 'parameters': {'batch_size': {'values': [16, 32, 64, 128, 256]},
                'batchnorm_3': {'values': [True, False]},
                'batchnorm_5': {'values': [True, False]},
                'dropout_2': {'values': [True, False]},
                'dropout_4': {'values': [True, False]},
                'dropout_6': {'values': [True, False]},
                'dropout_value': {'values': [0.3, 0.4, 0.5, 0.6]},
                'epochs': {'value': 10},
                'learning_rate': {'values': [0.0001,
                                             0.0005,
                                             0.001,
                                             0.005,
                                             0.01,
                                             0.05]},
                'n_features1': {'values': [128, 256, 512]},
                'n_features2': {'values': [128, 256, 512]},
                'optimizer'

### Обучение модели, запуски экспериментов

Простой тест на проверку правильности написанного кода

In [22]:
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

model = get_model(device, 0.3, False, False, False, True, True, 256, 128)
train_dataloader, test_dataloader = data_loaders(32)

metrics, all_losses, predicted_labels, true_labels = predict(model, test_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(predicted_labels, true_labels)
print("tests passed")

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/313 [00:00<?, ?it/s]

tests passed


Запустить обучение можно в ячейке ниже.

In [ ]:
wandb.agent(sweep_id, train, count=10) # this cell was done before, but my notebook decided to restart, so the output is empty

In [27]:
train()

### Проверка полученной accuracy

После всех экспериментов которые вы проделали, выберите лучшую из своих моделей, реализуйте и запустите функцию `evaluate`. Эта функция должна брать на вход модель и даталоадер с валидационными данными и возврашать accuracy, посчитанную на этом датасете.

In [46]:
#metrics, all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(val_dataset)
accuracy = accuracy_score(true_labels, predicted_labels)
print(accuracy)
print("Оценка за это задание составит {} баллов".format(min(5, 5 * accuracy / 0.44)))

0.5531
Оценка за это задание составит 5 баллов


### Отчёт об экспериментах 

Поскольку я использовала для проведения экспериментов wandb, я написала отчет там же, так как только там можно посмотреть графики: https://wandb.ai/checheanya/hw2/reports/Performance-comparison-HW2--VmlldzoxMjU4OTM4?accessToken=4tcfx7cbnpbfhcv86b9kjtccrnu572kb1o5p4iauki5cpgvphgqbwa4fkoacbq3c